In [1]:
# Import findspark and initialize.
!pip install findspark

import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, col
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [4]:
# 2. Create a temporary view of the DataFrame.

# Add the file from the URL to SparkFiles
spark.sparkContext.addFile(url)

# Read the CSV file into a DataFrame
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# Show the DataFrame to ensure the file is loaded correctly
df.show()

# Create a temporary view of the DataFrame
df.createOrReplaceTempView("home_sales")


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

query = """
SELECT
    date_built AS year_built,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    bedrooms = 4
GROUP BY
    date_built
ORDER BY
    date_built
"""

# Execute the query
result = spark.sql(query)

# Show the results
result.show()



+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|296800.75|
|      2011| 302141.9|
|      2012|298233.42|
|      2013|299999.39|
|      2014|299073.89|
|      2015|307908.86|
|      2016|296050.24|
|      2017|296576.69|
+----------+---------+



In [6]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

query = """
SELECT
    date_built AS year_built,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    bedrooms = 3 AND bathrooms = 3
GROUP BY
    date_built
ORDER BY
    date_built
"""

# Execute the query
result = spark.sql(query)

# Show the results
result.show()



+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [7]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

query = """
SELECT
    date_built AS year_built,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    bedrooms = 3
    AND bathrooms = 3
    AND floors = 2
    AND sqft_living >= 2000
GROUP BY
    date_built
ORDER BY
    date_built
"""

# Execute the query
result = spark.sql(query)

# Show the results
result.show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [8]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = """
SELECT
    view,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
GROUP BY
    view
HAVING
    AVG(price) >= 350000
ORDER BY
    view DESC
"""

# Execute the query
result = spark.sql(query)

# Show the results
result.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 1.4403088092803955 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.

spark.sql("CACHE TABLE home_sales")


DataFrame[]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

query = """
SELECT
    view,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
GROUP BY
    view
HAVING
    AVG(price) >= 350000
ORDER BY
    view DESC
"""
result_cached = spark.sql(query)

# Show the results
result_cached.show()

cached_runtime = time.time() - start_time
print("--- Cached query runtime: %s seconds ---" % cached_runtime)

# For comparison, run the same query without caching

spark.sql("UNCACHE TABLE home_sales")

# Start the timer for the uncached query
start_time = time.time()

# Execute the query without caching
result_uncached = spark.sql(query)

# Show the results
result_uncached.show()

# Stop the timer and calculate runtime
uncached_runtime = time.time() - start_time
print("--- Uncached query runtime: %s seconds ---" % uncached_runtime)

# Compare runtimes
print(f"Performance improvement: {uncached_runtime - cached_runtime:.2f} seconds")


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- Cached query runtime: 0.7785050868988037 seconds ---
+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- Uncached query runtime:

In [28]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

output_path = "/content/drive/MyDrive/Home_Sales_Partioned"

# Write the DataFrame as partitioned Parquet files
df.write.mode("overwrite").partitionBy("date_built").parquet(output_path)

print(f"Data successfully written to {output_path} in Parquet format, partitioned by 'date_built'.")


Data successfully written to /content/drive/MyDrive/Home_Sales_Partioned in Parquet format, partitioned by 'date_built'.


In [31]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
!ls /content/drive/MyDrive/Home_Sales_Partioned


'date_built=2010'  'date_built=2012'  'date_built=2014'  'date_built=2016'   _SUCCESS
'date_built=2011'  'date_built=2013'  'date_built=2015'  'date_built=2017'


In [16]:
# 11. Read the formatted parquet data.

# Read the Parquet data into a DataFrame
parquet_df = spark.read.parquet(output_path)

# Show the first few rows of the DataFrame
parquet_df.show()

# Print the schema to verify the structure
parquet_df.printSchema()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [17]:
# 12. Create a temporary table for the parquet data.

parquet_df.createOrReplaceTempView("parquet_home_sales")

# Verify the table creation by querying it
query_result = spark.sql("SELECT * FROM parquet_home_sales LIMIT 10")
query_result.show()


+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [20]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

query = """
SELECT
    view,
    ROUND(AVG(price), 2) AS avg_price
FROM
    parquet_home_sales
GROUP BY
    view
HAVING
    AVG(price) >= 350000
ORDER BY
    view DESC
"""

# Execute the query on the Parquet DataFrame
parquet_result = spark.sql(query)

# Show the results
parquet_result.show()

# Stop the timer and calculate runtime
parquet_runtime = time.time() - start_time
print("--- Parquet query runtime: %s seconds ---" % parquet_runtime)

# Compare with cached runtime

cached_runtime = 0.7785050868988037

# Compare runtimes
print(f"Performance difference: {parquet_runtime - cached_runtime:.2f} seconds")


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- Parquet query runtime: 0.8603589534759521 seconds ---
Performance difference: 0.08 seconds


In [21]:
# 14. Uncache the home_sales temporary table.

spark.sql("UNCACHE TABLE home_sales")


DataFrame[]

In [22]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' table cached? {is_cached}")


Is 'home_sales' table cached? False


In [34]:
# Zip the partitioned folder
!zip -r your_folder.zip /content/drive/MyDrive/Home_Sales_Partioned


  adding: content/drive/MyDrive/Home_Sales_Partioned/ (stored 0%)
  adding: content/drive/MyDrive/Home_Sales_Partioned/date_built=2010/ (stored 0%)
  adding: content/drive/MyDrive/Home_Sales_Partioned/date_built=2010/part-00000-bc585395-6e39-4280-9061-f4d9ebcaaba6.c000.snappy.parquet (deflated 24%)
  adding: content/drive/MyDrive/Home_Sales_Partioned/date_built=2010/.part-00000-bc585395-6e39-4280-9061-f4d9ebcaaba6.c000.snappy.parquet.crc (stored 0%)
  adding: content/drive/MyDrive/Home_Sales_Partioned/date_built=2011/ (stored 0%)
  adding: content/drive/MyDrive/Home_Sales_Partioned/date_built=2011/part-00000-bc585395-6e39-4280-9061-f4d9ebcaaba6.c000.snappy.parquet (deflated 24%)
  adding: content/drive/MyDrive/Home_Sales_Partioned/date_built=2011/.part-00000-bc585395-6e39-4280-9061-f4d9ebcaaba6.c000.snappy.parquet.crc (stored 0%)
  adding: content/drive/MyDrive/Home_Sales_Partioned/date_built=2012/ (stored 0%)
  adding: content/drive/MyDrive/Home_Sales_Partioned/date_built=2012/part-00